In [1]:
import numba
from numba import cuda
import numpy as np
import math

In [2]:
# CONSTANTS
ATOM_MAX = 29
N = int(1e4)

In [3]:
@cuda.jit
def compute_distance_mat(crds, natoms, dmat):
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bx = cuda.blockIdx.x
    
    dmat[bx, tx, ty] = 0.0
    
    # bounds check
    N = natoms[bx]
    if tx >= N: return
    if ty >= N: return 
    
    for i in range(3):
        dmat[bx, tx, ty] += ( crds[bx, tx, i] - crds[bx, ty, i] )**2
    dmat[bx, tx, ty] = math.sqrt(dmat[bx, tx, ty])  

In [4]:
crds = np.random.random((N, ATOM_MAX, 3))
crds_natoms = np.array([ATOM_MAX]*N)

In [5]:
# create result array on the GPU
shape = (N, ATOM_MAX, ATOM_MAX)
distance_mat_gpu = cuda.device_array(shape, dtype=np.float32)

In [6]:
%%timeit -n1 -r1 -o

# transfer structures data (the coordinates) to GPU
crds_gpu = cuda.to_device(crds)
crds_natoms_gpu = cuda.to_device(crds_natoms)

# launch kernel
compute_distance_mat[ N , (29, 29)](crds_gpu, crds_natoms_gpu, distance_mat_gpu)

# copy the data back
distance_mat_cpu = distance_mat_gpu.copy_to_host()

198 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 198 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [7]:
# store the timing result
GPU_TIMING = _

In [8]:
dmat_np_cpu = np.zeros((N, ATOM_MAX, ATOM_MAX))

In [9]:
%%timeit -n1 -r1 -o

for m in range(N):
    for i in range(ATOM_MAX):
        for j in range(ATOM_MAX):
            dmat_np_cpu[m,i,j] = np.sqrt(np.sum((crds[m, i] - crds[m,j])**2))

56.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<TimeitResult : 56.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [10]:
# store the timing result
CPU_TIMING = _

In [11]:
print('Speedup factor: ', CPU_TIMING.average / GPU_TIMING.average, 'X')

Speedup factor:  283.8613698743943 X


In [12]:
# copy the data back (again)
distance_mat_cpu = distance_mat_gpu.copy_to_host()

# check results
tol = 1e-4
error = 0
for m in range(N):
    for i in range(ATOM_MAX):
        for j in range(ATOM_MAX):
            r = dmat_np_cpu[m,i,j] - distance_mat_cpu[m,i,j]
            if r>tol:
                error=1
if not error:
    print('results agree')

results agree
